In [ ]:
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import integrate

import torch

In [ ]:
result_path = pathlib.Path.home()/'group'/'project'/'results'/'machine_learning'/'dnn'\
/'triggering'/'eval_outputs'/\
'signal_grid'/'220317_84to90_0to5cm_test_cnn_v2.npz'

data = pathlib.Path.home()/'group'/'project'/'results'/'beamforming'/'time_dependent'/\
'beamform_signals'/'220307_0to5cm_84to90_8192_gradb_corrected.npz'


result = np.load(result_path)

In [ ]:
output, target = result['output'], result['target']
output_softmax = torch.nn.functional.softmax(torch.tensor(output), dim=1).numpy()

In [ ]:
print(output)
print(torch.nn.functional.softmax(torch.tensor(output), dim=1).numpy())

In [ ]:
target_0 = np.argwhere(target == 0).squeeze()
target_1 = np.argwhere(target == 1).squeeze()

In [ ]:
target1_output = output[target_1, 1].reshape((6070, 100), order='F')
target0_output = output[target_0, 1]
print(target1_output.shape)
print(target0_output.shape)

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(target1_output.mean(axis=-1))
#ax.set_xlim(0, 500)

In [ ]:
signals, radius, pitch = np.load(data)['signals'], np.load(data)['radius'], np.load(data)['pitch']
where_not_zero = np.argwhere(abs(signals).sum(-1)>0).squeeze()

radius_not_zero = radius.flatten()[where_not_zero]
pitch_not_zero = pitch.flatten()[where_not_zero]

print(radius_not_zero.size)




# compare noise histogram to histogram of specific params

In [ ]:
select_radius = 0.01
select_pitch = 87.0
ind = np.argwhere(
    np.logical_and(radius_not_zero == select_radius, pitch_not_zero == select_pitch
                  )).squeeze()


In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

hist_noise = ax.hist(target0_output, 50, density=True)
hist_signal = ax.hist(target1_output[ind, :], 25, density=True)

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)


hist_out1 = ax.hist(output[target_1, 0], 50, density=True)
hist_out0 = ax.hist(output[target_0, 0], 50, density=True)
#hist_tar = ax.hist(, 50, density=True)

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)


hist_out1 = ax.hist(output_softmax[target_1, 0], 50, density=True)
hist_out0 = ax.hist(output_softmax[target_1, 1], 50, density=True)
#hist_tar = ax.hist(, 50, density=True)

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)


hist_out1 = ax.hist(output_softmax[target_1, 1], 50, density=True)
hist_out0 = ax.hist(output_softmax[target_0, 1], 50, density=True)
#hist_tar = ax.hist(, 50, density=True)

In [ ]:
bins = np.linspace(0., 1.01, 2048)

hist_signal = np.histogram(output_softmax[target_1, 1], bins, density=True)
hist_noise = np.histogram(output_softmax[target_0, 1], bins, density=True)

cdf_signal = np.zeros(hist_signal[0].size)
cdf_noise = np.zeros(hist_signal[0].size)

bins = bins[0:bins.size-1]

for i in range(hist_signal[0].size):
    
    #print(hist_signal[0][0:i+1], bins[1:i+1])
    cdf_signal[i] = integrate.trapezoid(hist_signal[0][0:i+1], bins[0:i+1], )
    cdf_noise[i] = integrate.trapezoid(hist_noise[0][0:i+1], bins[0:i+1], )

#cdf_signal = integrate.trapezoid(hist_signal[0], bins[1:], )

sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(cdf_signal)
ax.plot(cdf_noise)

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(1-cdf_noise, 1-cdf_signal)
#ax.plot(1-cdf_signal, 1-cdf_noise)
ax.plot(np.linspace(0, 1, 10000), np.linspace(0, 1, 10000))
ax.set_xscale('log')
ax.set_xlim(1e-3, 1)

# save roc curve results

In [ ]:
save_path = pathlib.Path.home()/'group'/'project'/'results'/'machine_learning'/\
'dnn'/'triggering'/'distributions'/'220317_dl_grid_87_100_cnn_v1_test_output_dist.npz'

np.savez(save_path,
         bins=bins,
         noise_pdf = hist_noise[0],
         signal_pdf = hist_signal[0],
         noise_cdf = cdf_noise, 
         signal_cdf = cdf_signal,
        )